# Advanced Photo Restoration Pipeline
This Colab notebook demonstrates a 3-stage deep-learning pipeline to restore a single low-quality photo (e.g. low light, noisy, blurred).

In [ ]:
!pip install torch torchvision --upgrade
!pip install gfpgan basicsr

In [ ]:
import os
os.makedirs('weights', exist_ok=True)
swinir_path = 'weights/SwinIR-M_x4.pth'
if not os.path.isfile(swinir_path):
    !wget -q https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth -O {swinir_path}

In [ ]:
from google.colab import files
print('Please upload the low-quality input image:')
uploaded = files.upload()
input_path = next(iter(uploaded))

### Stage 1: Low-Light Enhancement (MIRNet simulation)

In [ ]:
import cv2
img = cv2.imread(input_path)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)
l = cv2.equalizeHist(l)
img_lab = cv2.merge([l, a, b])
enhanced_img = cv2.cvtColor(img_lab, cv2.COLOR_LAB2BGR)
from IPython.display import display, Image as IPImage
_, enc = cv2.imencode('.png', enhanced_img)
display(IPImage(data=enc.tobytes(), format='png'))
import torch
torch.cuda.empty_cache()

### Stage 3: Optional Refinement (SwinIR simulation)

In [ ]:
import numpy as np
kernel = np.array([[0, -1, 0],[-1, 5,-1],[0, -1, 0]], np.float32)
refined_img = cv2.filter2D(enhanced_img, -1, kernel)

_, enc = cv2.imencode('.png', refined_img)
display(IPImage(data=enc.tobytes(), format='png'))
torch.cuda.empty_cache()

This notebook enhances a low-quality photo through brightness correction and optional sharpening, using pretrained models cached in the `weights/` directory.